In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

In [3]:
sys.path.append('../../')

In [4]:
from glob import glob
import os
import shutil
import time
import multiprocessing

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tqdm

from utils.parse_util import HTMLFileReader, LinkedInProfileParser

In [5]:
SOURCE_DATA_DIR = '/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/'

In [6]:
# profile_dirs = [x for x in os.listdir(DATA_DIR) if not x.endswith('.zip')]
# profile_dirs = sorted(profile_dirs, key=lambda x: int(x))

In [7]:
profile_dirs = os.listdir(SOURCE_DATA_DIR)

In [8]:
len(profile_dirs)

500

### Run first 8

In [9]:
def parse_profile(profile_path):
    html_reader = HTMLFileReader(profile_path)
    html_reader.load()
    clean_html = html_reader.get_clean_html_text()
    
    li_profile = LinkedInProfileParser(clean_html)
    li_profile.parse()
    li_profile_dict = li_profile.get_formatted_content()
    return li_profile_dict

In [10]:
LOG_DIR = ('/home/sjb/Projects/Research/\
LinkedIn_OB/data/sample_log/parallel/')

In [11]:
def write_logs(file_prefix, contents, log_dir=LOG_DIR):
    file_name = '{}logs.txt'.format(file_prefix)
    file_path = os.path.join(log_dir, file_name)
    
    content_str = '\n'.join(contents)
    
    with open(file_path, 'w') as f:
        f.write(content_str)
    

In [ ]:
def parse_profiles_for_directory(profile_dir):
    profile_fpaths = glob(os.path.join(profile_dir, '*.html'))
    log_file_prefix = os.path.basename(profile_dir) + '_multi_'
    to_log = []
    to_log.append('n_files: {}'.format(len(profile_fpaths)))
    
    tic = time.time()
    
    for fpath in profile_fpaths:
        try:
            parsed = parse_profile(fpath)
        except Exception as e:
            to_log.append('parse_error for: {}'.format(fpath))
    
    toc = time.time()
    to_log.append('total_time: {}'.format((toc - tic) / 60))
    
    write_logs(log_file_prefix, to_log, log_dir=LOG_DIR)
    print('Finished run for {}'.format(profile_dir))
    
    return
    

In [12]:
sample_profile_dir = '111'

sample_file_name = '5500168.html'

sample_file_path = os.path.join(SOURCE_DATA_DIR,
                                sample_profile_dir,
                                sample_file_name)

In [17]:
os.path.join(SOURCE_DATA_DIR, sample_profile_dir)

'/media/sjb/Sandra M/SJ_Partition/unzipped/First_1000_0/111'

In [14]:
html_reader = HTMLFileReader(sample_file_path)
html_reader.load()

In [13]:
try:
    parse_profile(sample_file_path)
except Exception as e:
    

ParserError: Document is empty

In [ ]:
N_PROCESS = 9

In [ ]:
p = multiprocessing.Pool(N_PROCESS)

In [ ]:
profile_dirs_full = [os.path.join(SOURCE_DATA_DIR, x) for x in profile_dirs]

In [ ]:
%%time

p.map(parse_profiles_for_directory, profile_dirs_full)

p.close()

In [ ]:
1000 / 24

In [ ]:
(2000 / 8) * 5 / 6